In [1]:
from MesoPy import Meso
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd

In [4]:
m = Meso(token='0191c61bf7914bd49b8bd7a98abb9469')
precip = m.precip(stid='MTM09', start='201504241800', end='201504271200', units='height|m')

In [5]:
a = m.timeseries('202002262339', '202003111605', stid = 'AMBC1', units = 'height|m')['STATION'][0]['OBSERVATIONS']['air_temp_set_1']
# 202002262339
# 202003111605
# AMBC1
b = [elem for elem in a if elem is not None]
np.nanmean(b)

1.7222852512155589

In [39]:
np.nanmean(np.array(a))

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [8]:
from glob import glob
import rasterio as rio

In [9]:
# glob('/scratch/zacharykeskinen/jpl/*/tiffs/*')
fp = '/bsuscratch/zacharykeskinen/data/uavsar/images/Lowman, CO/lowman_05208_21012-004_21015-009_0021d_s01_L090_01_int_grd/lowman_05208_21012-004_21015-009_0021d_s01_L090HV_01.cor.grd.tiff'
with rio.open(fp) as src:
    bounds  = src.bounds
    

bbox: list, optional
    Stations within a [lon/lat] box in the order [lonmin,latmin,lonmax,latmax] e.g. bbox="-120,40,-119,41"

In [12]:
for stat in m.metadata(start = '202002010000', end = '202002100000', bbox = bounds, vars = ['snow_depth','air_temp'])['STATION']:
    name = stat['NAME']
    long = stat['LONGITUDE']
    lat = stat['LATITUDE']
    elev = stat['ELEVATION']
    stid = stat['STID']
    status = stat['STATUS']
    print(name)
    print(stid)
    print(lat)
    print(long)
    print(stid)

Stanley, Stanley Ranger Station
KSNT
44.20861
-114.93444
KSNT
BOISE
BOII
43.60028
-116.17667
BOII
ATLANTA SUMMIT
ATAI1
43.75690
-115.23907
ATAI1
BANNER SUMMIT
BASI1
44.30342
-115.23447
BASI1
BOGUS BASIN
BOGI1
43.76377
-116.09685
BOGI1
COZY COVE
COZI1
44.28783
-115.65527
COZI1
GRAHAM GUARD STA.
GGSI1
43.95380
-115.27387
GGSI1
JACKSON PEAK
JKPI1
44.05092
-115.44322
JKPI1
MORES CREEK SUMMIT
MRKI1
43.93200
-115.66588
MRKI1
TRINITY MTN.
TRMI1
43.62903
-115.43818
TRMI1
BANNER SUMMIT
TMPE7
44.3
-115.23
TMPE7
BEARSKIN CREEK
BCFI1
44.385556
-115.5505
BCFI1
CANYON CREEK
TMPE8
44.23
-115.25
TMPE8
CLASS III 7-C
QEDA3
43.6
-116.21
QEDA3
CRANE CREEK
CRNI1
43.67
-116.13
CRNI1
CURLEW CREEK
CURI1
43.64
-116.13
CURI1
FIRE RAWS #1
CBPN5
43.6
-116.21
CBPN5
FIRE RAWS #2
SIGN5
43.6
-116.21
SIGN5
LITTLE ANDERSON
LTAI1
44.091111
-115.880556
LTAI1
LUCKY PEAK
LUFI1
43.5875
-115.9925
LUFI1
PINE CREEK
PNFI1
44.250278
-116.198611
PNFI1
TOWN CREEK
TCFI1
43.939583
-115.911528
TCFI1
CLASS III NPS7
TS321
43.5994
-116.

In [12]:
def raster_box_extract(img, x_coord, y_coord, box_side = 5):
    meta = img.meta

    # Use the transform in the metadata and your coordinates
    try:
        rowcol = rio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
        w = img.read(1, window=Window(rowcol[1], rowcol[0], box_side, box_side))
        return w
    except:
        pass
        # print('station outside bbox')

In [13]:
stat_ls = []
stid_ls = []
res = {}
for stat in m.metadata(start = start_flight, end = end_flight, bbox = bounds)['STATION']:
    long = float(stat['LONGITUDE'])
    lat = float(stat['LATITUDE'])
    with rio.open(fp) as src:
        w = raster_box_extract(src, long, lat)
    if len(w[~np.isnan(w)]) > 0:
        if stat['NAME'].lower().replace(' ','') not in stat_ls:
            obs = m.timeseries(start_flight, end_flight, stid = stat['STID'], vars = 'snow_depth')
            if obs:
                obs = obs['STATION'][0]['OBSERVATIONS']
                d = {}
                dt = pd.to_datetime(obs['date_time'])
                if 'snow_depth_set_1' in obs.keys():
                    d['datetime'] = dt
                    d['img_arr'] = w
                    d['snow_depth_set_1'] = obs['snow_depth_set_1']
                    for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
                        if anc_col in obs.keys():
                            d[anc_col] = obs[anc_col]
                if d:
                    d['elev'] = stat['ELEVATION']
                    d['lat'] = stat['LATITUDE']
                    d['long'] = stat['LONGITUDE']
                    d['tz'] = stat['TIMEZONE']
                    res[stat['NAME']] = d
            
            stat_ls.append(stat['NAME'].lower().replace(' ',''))

NameError: name 'start_flight' is not defined

In [14]:
plt.imshow(res['Treeline']['img_arr'])

KeyError: 'Treeline'

In [15]:
stat_ls = []
stid_ls = []
for stat in m.metadata(start = start_flight, end = end_flight, bbox = bounds)['STATION']:
    long = float(stat['LONGITUDE'])
    lat = float(stat['LATITUDE'])
    with rio.open(fp) as src:
        w = raster_box_extract(src, long, lat)
    if len(w[~np.isnan(w)]) > 0:
        if stat['NAME'].lower().replace(' ','') not in stat_ls:

            stat = m.timeseries(start_flight, end_flight, stid = stat['STID'])
            res = {}
            d = {}
            obs = stat['OBSERVATIONS']

            dt = pd.to_datetime(obs['date_time'])
            if 'snow_depth_set_1' in obs.keys():
                d['datetime'] = dt
                d['snow_depth_set_1'] = obs['snow_depth_set_1']
                for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
                    if anc_col in obs.keys():
                        d[anc_col] = obs[anc_col]
            if d:
                d['elev'] = stat['ELEVATION']
                d['lat'] = stat['LATITUDE']
                d['long'] = stat['LONGITUDE']
                d['tz'] = stat['TIMEZONE']
                res[stat['NAME']] = d
        stat_ls.append(stat['NAME'].lower().replace(' ',''))

NameError: name 'start_flight' is not defined

In [16]:
a = m.timeseries(start = start_flight, end = end_flight, stid = stid_ls)

res = {}
for stat in a['STATION']:
    # print(stat['SENSOR_VARIABLES'])
    d = {}
    obs = stat['OBSERVATIONS']

    dt = pd.to_datetime(obs['date_time'])
    if 'snow_depth_set_1' in obs.keys():
        d['datetime'] = dt
        d['snow_depth_set_1'] = obs['snow_depth_set_1']
        for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
            if anc_col in obs.keys():
                d[anc_col] = obs[anc_col]
    if d:
        d['elev'] = stat['ELEVATION']
        d['lat'] = stat['LATITUDE']
        d['long'] = stat['LONGITUDE']
        d['tz'] = stat['TIMEZONE']
        res[stat['NAME']] = d

NameError: name 'start_flight' is not defined

In [ ]:
for col in ann_df.columns:
    if 'acq' in col:
        print(col)

In [ ]:
ann_df = pd.read_csv('/home/zacharykeskinen/uavsar/data/slc_stack/lowman_23205_20002-007_20007-003_0013d_s01_L090_01_int_grd/lowman_23205_20002-007_20007-003_0013d_s01_L090_01_int_grd.csv')
start = pd.to_datetime(ann_df['stop time of acquisition for pass 1'][0])
end = pd.to_datetime(ann_df['start time of acquisition for pass 2'][0])

In [ ]:
def mesopy_date_parse(pd_date_str):
    return pd_date_str.strftime('%Y') + pd_date_str.strftime('%m') + pd_date_str.strftime('%d') + pd_date_str.strftime('%H') + pd_date_str.strftime('%M')

In [ ]:
mesopy_date_parse(pd.to_datetime('2020-10-04T20:30'))

In [ ]:
import datetime
isinstance((pd.to_datetime('2020-10-01')), datetime.datetime)


In [ ]:
start.month

In [ ]:
start_flight = '202001010000'
end_flight = '202001070000'
a = m.timeseries(start_flight, end_flight, stid = 'BASI1' )
dt = a['STATION'][0]['OBSERVATIONS']['date_time']
sd = a['STATION'][0]['OBSERVATIONS']['snow_depth_set_1']
swe = a['STATION'][0]['OBSERVATIONS']['snow_water_equiv_set_1']
temp = a['STATION'][0]['OBSERVATIONS']['air_temp_set_1']
a['STATION'][0]['OBSERVATIONS'].keys()

In [ ]:
from rasterio.windows import Window    
with rio.open(fp) as src:
    meta = src.meta

    # My target coordinates
    x_coord = -115.23447
    y_coord = 44.30342

    # Use the transform in the metadata and your coordinates
    rowcol = rio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)

    w = src.read(1, window=Window(rowcol[1], rowcol[0], 5, 5))
    # Do something with your data here.
    print(w)

In [ ]:
df =pd.read_csv('/home/zacharykeskinen/uavsar/results/test_wx.csv')

In [ ]:
plt.plot(df.set_index('Unnamed: 0')['BANNER SUMMIT']['snow_depth_set_1'])

In [ ]:
import pickle
with open('/home/zacharykeskinen/uavsar/results/test_wx.pkl', 'rb') as f:
    res  = pickle.load(f)

In [ ]:
plt.scatter(res['Lower Weather']['snow_depth_set_1'])

In [ ]:
for site,obs in res.items():
    print(site)
    #print(obs)
    plt.scatter(obs['datetime'], obs['snow_depth_set_1'])
    plt.show()

In [8]:
import pandas as pd
import numpy as np
df = pd.DataFrame([0,-1,-2,3,4,None,np.nan], index = pd.date_range(start = '20210101',end = '20210201', periods = 7))
df = df.resample('D').mean()

In [18]:
df[(df[0].notnull()) & (df[0]>0)]

,0
2021-01-16,3.0
2021-01-21,4.0
